Este código é a implementação de um modelo de machine learning usado em uma competição de ciência de dados de uma empresa real, para classificar funcionários inadimplentes, para concessão de crédito.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter


path = '/content/default_train_set.csv'

df = pd.read_csv(path)

df_test = pd.read_csv('/content/Default_Test_Set.csv')

df_economic = pd.read_csv('/content/economic_indicators.csv')

Separando o dataset entre teste e treino

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

# Divisão
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


convertendo variáveis de data para datetime.

In [ ]:
X_train['disbursement_date'] = pd.to_datetime(X_train['disbursement_date'])
X_train['due_date'] = pd.to_datetime(X_train['due_date'])
X_test['disbursement_date'] = pd.to_datetime(X_test['disbursement_date'])
X_test['due_date'] = pd.to_datetime(X_test['due_date'])

df_test['disbursement_date'] = pd.to_datetime(df_test['disbursement_date'])
df_test['due_date'] = pd.to_datetime(df_test['due_date'])

df_test['disbursement_date'] = df_test['disbursement_date'].apply(lambda date: date.toordinal())
df_test['due_date'] = df_test['due_date'].apply(lambda date: date.toordinal())

X_train['disbursement_date'] = X_train['disbursement_date'].apply(lambda date: date.toordinal())
X_train['due_date'] = X_train['due_date'].apply(lambda date: date.toordinal())
X_test['disbursement_date'] = X_test['disbursement_date'].apply(lambda date: date.toordinal())
X_test['due_date'] = X_test['due_date'].apply(lambda date: date.toordinal())

aplicando feature engineering:

# 1. data['Repayment_Percentage'] = data['Total_Amount_to_Repay'] / data['Total_Amount']

Objetivo: Calcula a porcentagem do valor total que precisa ser reembolsado em relação ao valor total do empréstimo.
Razão: Essa feature indica o peso do reembolso em relação ao valor original do empréstimo, o que pode ser um indicador importante de risco de inadimplência. Um valor mais alto sugere um custo maior para o tomador do empréstimo.
# 2. data['Loan_Duration_to_Amount_Ratio'] = data['duration'] / data['Total_Amount']

Objetivo: Calcula a relação entre a duração do empréstimo e o valor total do empréstimo.
Razão: Essa feature busca capturar como a duração do empréstimo se relaciona com o valor total. Empréstimos de maior valor com prazos mais curtos podem representar um risco maior, enquanto empréstimos de menor valor com prazos longos podem ter um perfil de risco diferente.

# 3. data['Profit_Margin'] = data['Total_Amount_to_Repay'] - data['Total_Amount']

Objetivo: Calcula a margem de lucro do empréstimo, que é a diferença entre o valor total a ser reembolsado e o valor total do empréstimo.
Razão: Essa feature representa o lucro obtido com o empréstimo. Uma margem de lucro maior pode indicar um empréstimo mais lucrativo, mas também pode estar associada a um risco maior de inadimplência.

# 4. data['Adjusted_Profit_Margin'] = data['Profit_Margin'] / data['duration']

Objetivo: Calcula a margem de lucro ajustada pela duração do empréstimo.
Razão: Essa feature busca normalizar a margem de lucro, levando em consideração o tempo do empréstimo. Isso permite comparar a lucratividade de empréstimos com durações diferentes.

# 5. data['Lender_Contribution_Ratio'] = data['Amount_Funded_By_Lender'] / data['Total_Amount']

Objetivo: Calcula a proporção do valor total do empréstimo que foi financiada pelo credor.
Razão: Essa feature indica o nível de participação do credor no empréstimo. Um valor mais alto sugere um maior comprometimento do credor com o empréstimo.

In [ ]:
def feature_engineering(data):
    data['Repayment_Percentage'] = data['Total_Amount_to_Repay'] / data['Total_Amount']
    data['Loan_Duration_to_Amount_Ratio'] = data['duration'] / data['Total_Amount']
    data['Profit_Margin'] = data['Total_Amount_to_Repay'] - data['Total_Amount']
    data['Adjusted_Profit_Margin'] = data['Profit_Margin'] / data['duration']
    data['Lender_Contribution_Ratio'] = data['Amount_Funded_By_Lender'] / data['Total_Amount']

    # Conversões de datas
    data['disbursement_date'] = pd.to_datetime(data['disbursement_date'])
    data['due_date'] = pd.to_datetime(data['due_date'])
    data['Days_to_Due'] = (data['due_date'] - data['disbursement_date']).dt.days

    return data

# Aplica separadamente para os dois conjuntos
X_train = feature_engineering(X_train)
X_test = feature_engineering(X_test)

df_test = feature_engineering(df_test)


Dropando colunas que não serão usadas


In [ ]:
X_train.drop(columns=['ID','country_id', 'customer_id','tbl_loan_id','lender_id',
                          'New_versus_Repeat'], inplace=True)

X_test.drop(columns=['ID','country_id', 'customer_id','tbl_loan_id','lender_id',
                          'New_versus_Repeat'], inplace=True)

In [ ]:
df_test.drop(columns=['country_id', 'customer_id','tbl_loan_id','lender_id',
                          'New_versus_Repeat'], inplace=True)

categorizando a variavel lender_portion: a fração do valor total de um empréstimo que é financiada por um credor específico.

e categorizando o repayment_percentage: representa a proporção do valor total que precisa ser reembolsado em relação ao valor total do empréstimo.

In [ ]:
def categorize_lender_portion(portion):
    if portion > 0.5:
        return 5
    elif portion == 0.5:
        return 4
    elif portion < 0.5 and portion >= 0.3:
        return 3
    elif portion < 0.3 and portion >= 0.2:
        return 2
    elif portion < 0.2 and portion > 0.0:
        return 1
    elif portion == 0.0:
        return 0

X_train['Lender_portion_category'] = X_train['Lender_portion_Funded'].apply(categorize_lender_portion)
X_test['Lender_portion_category'] = X_test['Lender_portion_Funded'].apply(categorize_lender_portion)
df_test['Lender_portion_category'] = df_test['Lender_portion_Funded'].apply(categorize_lender_portion)

# Categorizar Repayment_Percentage
X_train['Repayment_Percentage'] = X_train['Total_Amount_to_Repay'] / X_train['Total_Amount']
X_test['Repayment_Percentage'] = X_test['Total_Amount_to_Repay'] / X_test['Total_Amount']
df_test['Repayment_Percentage'] = df_test['Total_Amount_to_Repay'] / df_test['Total_Amount']

def categorize_repayment_percentage(percentage):
    if percentage > 1.5:
        return 3
    elif percentage > 1.2:
        return 2
    elif percentage > 1.0:
        return 1
    else:
        return 0

X_train['Repayment_Category'] = X_train['Repayment_Percentage'].apply(categorize_repayment_percentage)
X_test['Repayment_Category'] = X_test['Repayment_Percentage'].apply(categorize_repayment_percentage)
df_test['Repayment_Category'] = df_test['Repayment_Percentage'].apply(categorize_repayment_percentage)

X_train = pd.get_dummies(X_train, columns=['Lender_portion_category', 'Repayment_Category'], prefix=['LenderCat', 'RepayCat'])
X_test = pd.get_dummies(X_test, columns=['Lender_portion_category', 'Repayment_Category'], prefix=['LenderCat', 'RepayCat'])
df_test = pd.get_dummies(df_test, columns=['Lender_portion_category', 'Repayment_Category'], prefix=['LenderCat', 'RepayCat'])

X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

Fazendo um experimento. Criando uma sub amostragem no dataset da categoria alvo. para colocar as duas categorias com mesmo numero de entradas, a custo de cortar entradas da classe com maior amostragem. o experimento não funcionou bem, pois informações significativas da classe maior foram perdidas. não utilizei este método, mas deixei para fins de documentação.

In [ ]:
from sklearn.utils import resample

# Função para subsamplear
def balanced_subsample(X, y):

    # Concatenar X_train e y_train para processar juntos
    data = pd.concat([X, y], axis=1)

    # Identificar as classes no y_train
    min_class = y.value_counts().min()  # Pega o menor tamanho das classes

    # Subsamplear cada classe para ter o mesmo tamanho
    sampled = pd.concat([
        resample(data[data[y.name] == label],  # Subsamplear para cada classe
                 replace=False,               # Não queremos duplicar dados
                 n_samples=min_class,         # Número de amostras para igualar as classes
                 random_state=42)             # Controlar a aleatoriedade
        for label in y.unique()
    ])

    X_balanced = sampled.drop(columns=y.name)
    y_balanced = sampled[y.name]
    return X_balanced, y_balanced

X_train_balanced, y_train_balanced = balanced_subsample(X_train, y_train)

# Verificar os tamanhos balanceados
print(f"Tamanhos após o subsampleamento:\n{y_train_balanced.value_counts()}")


Tamanhos após o subsampleamento:
target
0    1006
1    1006
Name: count, dtype: int64


criando variaveis contendo apenas numeros para o pre processamento.

In [ ]:
num_var = X_train.select_dtypes(include=np.number).columns
num_var_test = X_test.select_dtypes(include=np.number).columns
num_var_df_test = df_test.select_dtypes(include=np.number).columns

escalonamento padrão para as variaveis numéricas


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train[num_var])

X_test_scaled = scaler.transform(X_test[num_var_test])

df_test_scaled = scaler.transform(df_test[num_var_test])


implementando o modelo random forest depois do pré processamento.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
import numpy as np

# Definir os hiperparâmetros a testar
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Inicializando o modelo de Random Forest
rf = RandomForestClassifier(random_state=42)

# Realizando o Random Search com 5-fold cross-validation
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=8, cv=5, verbose=1, random_state=42, n_jobs=-1
)

# Treinando o modelo
random_search.fit(X_train_scaled, y_train)

# Melhores parâmetros encontrados
print(f"Melhores parâmetros: {random_search.best_params_}")

# Avaliação no conjunto de teste
y_pred = random_search.predict(X_test_scaled)

# Mostrar o F1 Score
print(f"F1 Score no conjunto de teste: {f1_score(y_test, y_pred)}")

# Imprimir classificação detalhada
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
5 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_pa

Melhores parâmetros: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
F1 Score no conjunto de teste: 0.8549019607843137
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13479
           1       0.84      0.87      0.85       252

    accuracy                           0.99     13731
   macro avg       0.92      0.93      0.93     13731
weighted avg       0.99      0.99      0.99     13731



a acurácia deu muito alta, e não é métrica para se observar em um problema como esse, e sim a métrica precisão e recall. Isso porque as classes estão extremamente desbalanceadas, o que é normal em um dataset de indadimplencia ou, de fraudes. Há muito menos indadimplentes do que adimplentes normalmente.
Por isso, o que estamos avaliando aqui é o precision e recall, que, para este dataset, foi de 0.84 e 0.87.  

Aqui embaixo escolhi os melhores parametros para o random forest de forma aleatória a partir de um escopo pré definido de parâmetros. O algoritmo escolhe o melhor parametro do escopo para eu utilizar.

In [ ]:
rf_best_params = {
    'n_estimators': 50,
    'min_samples_split': 5,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'max_depth': None,
    'bootstrap': False
}

# Inicializando o modelo Random Forest com os melhores parâmetros
rf_best = RandomForestClassifier(**rf_best_params, random_state=42)

# Treinando o modelo nos dados de treinamento
rf_best.fit(X_train_scaled, y_train)

# Fazendo predições no conjunto de teste
y_pred_3 = rf_best.predict(X_test_scaled)

# Avaliando o modelo com F1 Score
f1 = f1_score(y_test, y_pred_3)
print(f"F1 Score no conjunto de teste: {f1}")

# Imprimir classificação detalhada
print(classification_report(y_test, y_pred_3))

F1 Score no conjunto de teste: 0.8549019607843137
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13479
           1       0.84      0.87      0.85       252

    accuracy                           0.99     13731
   macro avg       0.92      0.93      0.93     13731
weighted avg       0.99      0.99      0.99     13731



Aqui preparo o modelo para ser exportado para fazer previsões com mais dados, com o objetivo de testar a robustez do modelo

In [ ]:
X_test_comp = df_test_scaled
predictions = rf_best.predict(X_test_comp)

In [ ]:
submission_1 = pd.DataFrame({
    'ID': df_test['ID'],
    'target': predictions
})

submission_1.to_csv('submission_1.csv', index=False)